In [81]:
import findspark
findspark.init()

In [82]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [83]:
spark=SparkSession.builder.appName('popular movie').getOrCreate()

In [84]:
sc=spark.sparkContext

In [85]:
def mapper(element):
    element=element.split('\t')
    return Row(userId=element[0],movieId=element[1],rating=int(element[2]),timestamp=element[3])

In [86]:
def movieDict():
    di=dict()
    with open("/Users/Prince.Bharti/Desktop/data/spark-course/ml-100k/u.item",encoding='latin-1') as file:
        line=file.readline()
        while(len(line)>0):
            li=line.split('|')
            di[li[0]]=li[1]
            line=file.readline()
    return di

In [87]:
base_rdd=sc.textFile("file:///Users/Prince.Bharti/Desktop/data/spark-course/ml-100k/u.data")
broadcast_variable=sc.broadcast(movieDict())

In [88]:
cleaned_rdd=base_rdd.map(mapper)

In [89]:
cleaned_rdd.first()

Row(movieId='242', rating=3, timestamp='881250949', userId='196')

In [90]:
df=spark.createDataFrame(cleaned_rdd)

In [91]:
#result=df.groupBy('movieId').sum('rating').orderBy('sum(rating)',ascending=False)
result=df.groupBy('movieId').count().orderBy('count',ascending=False)



In [92]:
result=result.collect()

In [93]:
for i in result[:10]:
    print('most popular movie is {} with total rating of {}'.format(broadcast_variable.value[i[0]],i[1]))

most popular movie is Star Wars (1977) with total rating of 583
most popular movie is Contact (1997) with total rating of 509
most popular movie is Fargo (1996) with total rating of 508
most popular movie is Return of the Jedi (1983) with total rating of 507
most popular movie is Liar Liar (1997) with total rating of 485
most popular movie is English Patient, The (1996) with total rating of 481
most popular movie is Scream (1996) with total rating of 478
most popular movie is Toy Story (1995) with total rating of 452
most popular movie is Air Force One (1997) with total rating of 431
most popular movie is Independence Day (ID4) (1996) with total rating of 429


In [79]:
spark.stop()